In [36]:
import numpy as np
import pandas as pd
import re, nltk
import pickle

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

import string

from nltk.stem import WordNetLemmatizer
from nltk import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import sklearn

stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
             "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
             'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 
             'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
             'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
             'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 
             'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 
             'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
             'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 
             'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 
             'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', 
             "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 
             "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', 
             "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 
             'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
             'wouldn', "wouldn't","say"]


def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

tokenizer = RegexpTokenizer(r'\w+')

cv= CountVectorizer(lowercase=True, max_df=.90, min_df=3, stop_words='english')

In [51]:
words=['#Agent','#Customer','ameriprise','hi','ok','im','account','client','group','um','umm','unk','sir','maam','zero','one','two','three','four','five','six','seven','eight','nine']

In [38]:
Base_Data = pd.read_csv(r"C:\Users\aprabh13\OneDrive - Ameriprise\NLP\Speech_Data\Top_10000.csv")
Base_Data.head()


,Transcript,HDS Call Index
0,#Agent : THANK YOU FOR CALLING <UNK> MY NAME I...,1530451759
1,#Agent : BROKERAGE ANY BUSINESS MY NAME IS SHE...,1530451784
2,#Agent : [LAUGHTER] THANK YOU FOR CALLING AMER...,1530451816
3,#Customer : MY SERVICE // #Agent : THANK YOU F...,1530451805
4,#Agent : THANK YOU FOR CALLING OUTGOING TRANSF...,1530451879


In [39]:
Base_Data=Base_Data[['Transcript']]
Base_Data.rename(columns={Base_Data.columns[0]:"Data"},inplace=True)
Base_Data.head()

,Data
0,#Agent : THANK YOU FOR CALLING <UNK> MY NAME I...
1,#Agent : BROKERAGE ANY BUSINESS MY NAME IS SHE...
2,#Agent : [LAUGHTER] THANK YOU FOR CALLING AMER...
3,#Customer : MY SERVICE // #Agent : THANK YOU F...
4,#Agent : THANK YOU FOR CALLING OUTGOING TRANSF...


In [40]:
def preprocess(text):
    data =''
    for token in tokenizer.tokenize(text):
        token=token.lower()
        token=WordNetLemmatizer().lemmatize(token,pos='v')
        if token not in stopwords :
            data=data+(token)+' '
    return data

In [41]:
def pos_tagging(text):
    tags=nltk.pos_tag(tokenizer.tokenize(text))
    noun_verbs = [word for word,pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' or pos == 'VB' or pos == 'VBD' or pos == 'VBG' or pos == 'VBN'or pos == 'VBP' or pos == 'VBZ')]
    data=''
    for word in noun_verbs:
        data=data+word+' '
    return data   

In [52]:
def Data_prep_LDA(Data,Special_words="",NV_only=0):
    global dtm,Data_preprocess
    stopwords.extend(Special_words)
    Data_wo_punc=Data['Data'].map(remove_punctuations)
    Data_preprocess=Data_wo_punc.map(preprocess)
    if NV_only==1:
        Data_preprocess=Data_preprocess.map(pos_tagging)
    dtm = cv.fit_transform(Data_preprocess)
    
    perplexity_values = []
    topic = []

    for x in range (10,13):      
        topic.append(x)
        LDA = LatentDirichletAllocation(n_components=x,random_state=420,doc_topic_prior=0.1,topic_word_prior=0.1)
        LDA.fit(dtm)
        z=LDA.perplexity(dtm)
        perplexity_values.append(z)
        
    return topic,perplexity_values

In [53]:
topic,perplexity=Data_prep_LDA(Base_Data,words)

In [54]:
print(topic,perplexity)

[10, 11, 12] [640.1283934419564, 639.784189925023, 635.7774856884248]


In [55]:
def LDA(dtm,n_topics):
    global LDA_model,Data_preprocess
    LDA_model = LatentDirichletAllocation(n_components=n_topics,random_state=420,doc_topic_prior=0.1,topic_word_prior=0.1,
                                      learning_method='online')
    LDA_model.fit(dtm)
    lda_output = LDA_model.transform(dtm)

    topicnames = ["Topic" + str(i) for i in range(LDA_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(Data_preprocess))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic
    return df_document_topic

In [56]:
doc_topic=LDA(dtm,12)

In [57]:
doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,dominant_topic
Doc0,0.0,0.00,0.0,0.00,0.62,0.00,0.00,0.00,0.00,0.14,0.23,0.00,4
Doc1,0.0,0.02,0.0,0.00,0.23,0.00,0.11,0.50,0.00,0.13,0.00,0.00,7
Doc2,0.0,0.00,0.0,0.23,0.39,0.18,0.00,0.17,0.00,0.00,0.00,0.02,4
Doc3,0.0,0.00,0.0,0.00,0.85,0.00,0.00,0.08,0.00,0.05,0.00,0.00,4
Doc4,0.0,0.24,0.0,0.00,0.39,0.00,0.11,0.00,0.00,0.25,0.00,0.00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc9995,0.0,0.42,0.0,0.00,0.00,0.00,0.06,0.00,0.00,0.50,0.00,0.00,9
Doc9996,0.0,0.00,0.0,0.00,0.00,0.12,0.03,0.04,0.00,0.80,0.00,0.00,9
Doc9997,0.0,0.00,0.0,0.03,0.82,0.00,0.00,0.00,0.03,0.12,0.00,0.00,4
Doc9998,0.0,0.00,0.0,0.10,0.48,0.00,0.00,0.17,0.02,0.17,0.06,0.00,4


In [58]:
def show_top_words():
    for index,topic in enumerate(LDA_model.components_):
        print(f'THE TOP 10 WORDS FOR TOPIC #{index}')
        print([cv.get_feature_names()[i] for i in topic.argsort()[-10:]])
        print('\n')


In [59]:
show_top_words()

THE TOP 10 WORDS FOR TOPIC #0
['clark', 'assurance', 'rebate', 'dispute', 'events', 'tracy', 'reward', 'maryland', 'calendar', 'fail']


THE TOP 10 WORDS FOR TOPIC #1
['beneficiary', 'receive', 'claim', 'thats', 'address', 'know', 'need', 'form', 'letter', 'send']


THE TOP 10 WORDS FOR TOPIC #2
['xray', 'assets', 'heather', 'bruce', 'exclusive', 'melissa', 'diana', 'travel', 'privilege', 'mary']


THE TOP 10 WORDS FOR TOPIC #3
['team', 'sure', 'try', 'like', 'help', 'advisor', 'yeah', 'id', 'service', 'mhm']


THE TOP 10 WORDS FOR TOPIC #4
['right', 'check', 'let', 'need', 'thats', 'information', 'dont', 'mhm', 'yes', 'number']


THE TOP 10 WORDS FOR TOPIC #5
['try', 'youre', 'look', 'gonna', 'dont', 'thats', 'right', 'like', 'yeah', 'know']


THE TOP 10 WORDS FOR TOPIC #6
['come', 'cash', 'broker', 'party', 'fund', 'money', 'company', 'service', 'brokerage', 'transfer']


THE TOP 10 WORDS FOR TOPIC #7
['mhm', 'let', 'need', 'like', 'right', 'look', 'thats', 'yeah', 'form', 'case']


